In [2]:
# import packages and libraries
import pandas as pd
import lightgbm
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn.metrics import mean_squared_error, explained_variance_score
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools
import uuid
import os
init_notebook_mode(connected=True)
from sklearn.neighbors import KNeighborsRegressor
from modAL.models import ActiveLearner
import random
import sys
from random import randint
import pickle

In [3]:
random.seed(123456)

In [4]:
from random import seed
from random import randint
# seed random number generator
seed(1)
randint(0, 4292967297)

577090037

In [5]:
#ensure repetabillity
seed = random.randrange(sys.maxsize)
random.seed(123456)
seed_list = []
for i in range(20):
    seed_list.append(randint(0, 4292967297))
#seed_list

In [26]:
###help functions###
def PlotCVRes_plotly(X, y, model, cv, scoring, titel):
    dir = os.path.dirname(os.path.abspath("__file__"))
    id = uuid.uuid4().hex
    if titel == '': 
        titel = str(model).replace('\n', '').replace('  ', '')
        titel_new = ''
        pos=0
        com_positions = pd.DataFrame([i for i, ltr in enumerate(titel) if (ltr == ',')])
        split_positions = [com_pos[0] for ind, com_pos in com_positions.iterrows() if ((ind+1)%5==0)] + [len(titel)]
        for ind in split_positions:
            titel_new = titel_new + ' \n ' + titel[pos:ind]
            pos = ind
        titel = titel_new[3:]

    cv_scores = cross_validate(model, np.array(X), np.array(y), cv=cv, scoring=scoring)
    print('MAE_test_mean: ' + str(round(abs(cv_scores['test_MAE']).mean(), 2)) + ', MAE_test_std: ' + str(
        round(abs(cv_scores['test_MAE']).std(), 2)))
    print('RMSE_test_mean: ' + str(round(np.sqrt(abs(cv_scores['test_RMSE'])).mean(), 2)) + ', RMSE_test_std: ' + str(
        round(np.sqrt(abs(cv_scores['test_RMSE'])).std(), 2)))
    print('R2_test_mean: ' + str(round(cv_scores['test_R2'].mean(), 3)) + ', R2_test_std: ' + str(
        round(cv_scores['test_R2'].std(), 3)))
    
    predicted = cross_val_predict(model, X, y, cv=cv)
    #predicted/real plot
    trace1 = go.Scatter(
        x = [y.min(), y.max()],
        y = [y.min(), y.max()],
        name = 'Ideal prediction line',
        line = dict(
            color = ('rgb(0, 0, 0)'),
            width = 3,
            dash = 'dash')
    )
    trace2 = go.Scattergl(
            x = y,
            y = predicted,
            name = 'Model Predictions',
            mode = 'markers',
            marker= dict(size= 3,
                opacity= 0.7,
                color = ('rgb(0, 102, 204)')
               ),
            #text=item_pairs,
            #name=orders,
            #hoverinfo = 'all', #"text + name",
            )

    data = [trace2, trace1]
    
    
    layout = go.Layout(
        xaxis=dict(
            title= 'Real Target Values',
            #showgrid=True,
            #zeroline=True,
            #showline=True,
        ),
        yaxis=dict(
            title= 'Predicted Target Values',
            #showgrid=True,
            #zeroline=True,
            #showline=True,
        ),
        # title="ML Model Cross-validation Results",
        font=dict(family='arial', size=20),
        autosize=False,
        width=600,
        height=600,
        showlegend=False,
        hovermode = "closest",
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, 'pred_real_scatter')

def Model_validation(X, y, model, scoring, data_stand=False, cv=5, plotCV=True, titel=''):
    if data_stand:
        Xs = StandardScaler().fit_transform(X)
    else:
        Xs = X
    # Plotting Cross-Validated Predictions
    if plotCV:
        PlotCVRes_plotly(Xs, y, model, cv, scoring, titel)
    else:
        cv_scores = cross_validate(model, Xs, y, cv=cv, scoring=scoring)
        print('MAE_test_mean: ' + str(abs(cv_scores['test_MAE']).mean()) + ', MAE_test_std: ' + str(
            abs(cv_scores['test_MAE']).std()))
        print('RMSE_test_mean: ' + str(np.sqrt(abs(cv_scores['test_RMSE'])).mean()) + ', RMSE_test_std: ' + str(
            np.sqrt(abs(cv_scores['test_RMSE'])).std()))
        print('R2_test_mean: ' + str(cv_scores['test_R2'].mean()) + ', R2_test_std: ' + str(
            cv_scores['test_R2'].std()))

In [23]:
data_path_ = r"..\..\01_Data\PUMA_DYNamics\pumadyn-8nm\Dataset.data"
data = pd.read_csv(data_path_, header=None, sep='  ')#.loc[::200,:]
#data.columns=cols
data.shape
#data.head()

C:\ProgramData\Anaconda3\envs\active_ml\lib\site-packages\ipykernel_launcher.py:2: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



(8192, 9)

In [13]:
input_file = '../../01_Data/pumadyn_8nm_ML.txt'
file_object = open(input_file, 'rb')
str_mdl = file_object.read()
reference_regr = pickle.loads(str_mdl)

[19:32:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [27]:
# set specific paths
scoring = {'R2': 'r2', 'RMSE': 'neg_mean_squared_error', 'MAE': 'neg_mean_absolute_error'}

X, y = data.iloc[:, :-1], data.iloc[:, -1]

# Shuffel data
np.random.seed(1337)
rng_state = np.random.get_state()
X = np.random.permutation(X)
np.random.set_state(rng_state)
y = np.random.permutation(y)
y=y.ravel()

xgbParams = {'learning_rate': 0.064, 
              'n_estimators': 1000, 
              'n_jobs': -1}
model = XGBRegressor()

xgbParams = {'learning_rate': 0.024,
             'n_estimators': 580,
             'max_depth': 6,
             'missing': -1,
             'tree_method': 'hist',
             'min_child_weight': 1,
             'gamma': 0,
             'subsample': 0.7,
             'colsample_bytree': 0.6,
             'reg_alpha': 1,
             'scale_pos_weight': 1,
             'random_state': 736283,
             'n_jobs': -1}
model = XGBRegressor(**xgbParams)

lgbmParams = {'learning_rate': 0.064, 
            'n_estimators': 456, 
            'num_leaves': 97, 
            'n_jobs': -1,
            'objective': 'regression'}

model = lightgbm.LGBMRegressor(**lgbmParams)

Model_validation(X, y, model, scoring=scoring,
                        cv=10, data_stand=False, 
                        plotCV=True, titel='')

MAE_test_mean: 0.82, MAE_test_std: 0.02
RMSE_test_mean: 1.09, RMSE_test_std: 0.02
R2_test_mean: 0.96, R2_test_std: 0.002


In [541]:
data.shape

(8192, 9)

In [1224]:
# set specific paths
scoring = {'R2': 'r2', 'RMSE': 'neg_mean_squared_error', 'MAE': 'neg_mean_absolute_error'}

X, y = data.iloc[:, :-1], data.iloc[:, -1]

# Shuffel data
np.random.seed(1337)
rng_state = np.random.get_state()
X = np.random.permutation(X)
np.random.set_state(rng_state)
y = np.random.permutation(y)
y=y.ravel()

lgbmParams = {'boosting_type': 'gbdt',
                'n_estimators': 2500,
                'objective': 'regression',
                'learning_rate': 0.1,
                'max_depth': 8,
                'num_leaves': 30,
                'n_jobs': -1
              }

lgbmParams = {'boosting_type': 'gbdt',
              'random_state': 736283,
              'num_leaves': 400,
              'min_data_in_leaf': 5,
              'max_bin': 500,
              'learning_rate': 0.08,
              'n_estimators': 771,
              'n_jobs': -1,
              'subsample': 0.8
              }

# lgbmParams = {'boosting_type': 'gbdt',
#               'random_state': 736283,
#               'num_leaves': 40,
#               'min_data_in_leaf': 5,
#               'max_bin': 500,
#               'learning_rate': 0.08,
#               'n_estimators': 77,
#               'n_jobs': -1,
#               'subsample': 0.8
#               }

lgbmParams = {'learning_rate': 0.045, 
              'n_estimators': 581, 
              'num_leaves': 84, 
              'objective': 'regression'}

lgbmParams = {#'learning_rate': 0.1, 
              'n_estimators': 100, 
              'objective': 'regression'}

model = lightgbm.LGBMRegressor(**lgbmParams)

xgbParams = {'learning_rate': 0.024,
             'n_estimators': 4900,
             'max_depth': 6,
             'missing': -1,
             'tree_method': 'hist',
             'min_child_weight': 1,
             'gamma': 0,
             'subsample': 0.7,
             'colsample_bytree': 0.6,
             'reg_alpha': 1,
             'scale_pos_weight': 1,
             'random_state': 736283,
             'n_jobs': -1}
xgbParams = {'learning_rate': 0.045, 
              'n_estimators': 581, 
              'n_jobs': -1}
model = XGBRegressor(**xgbParams)

#model = KNeighborsRegressor(n_neighbors=10, weights='uniform')

Model_validation(X, y, model, scoring=scoring,
                        cv=10, data_stand=False, 
                        plotCV=True, titel='')

[18:39:36] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:39:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:39:39] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:39:40] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:39:41] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:39:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:39:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:39:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:39:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

In [543]:
# import pickle
# model.fit(X, y)
# str_mdl = pickle.dumps(model, 0).decode() # dump to model in a string
# #restored_mdl=pickle.loads(pickled_mdl.encode()) #read str object to ml model
# file_object = open('pumadyn_8nm_ML.txt', 'w')
# file_object.write(str_mdl)

# #Read models from string
# file_object = open('pumadyn_8nm_ML.txt', 'r')
# str_mdl = file_object.read()
# pickle.loads(str_mdl.encode())

In [544]:
import pickle
model.fit(data.iloc[:, :-1], data.iloc[:, -1])
str_mdl = pickle.dumps(model, 0) # dump to model in a string
file_object = open('pumadyn_8nm_ML.txt', 'wb')
file_object.write(str_mdl)

#Read models from string
file_object = open('pumadyn_8nm_ML.txt', 'rb')
str_mdl = file_object.read()
pickle.loads(str_mdl)

C:\ProgramData\Anaconda3\envs\active_ml\lib\site-packages\xgboost\core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

C:\ProgramData\Anaconda3\envs\active_ml\lib\site-packages\xgboost\core.py:588: FutureWarning:

Series.base is deprecated and will be removed in a future version



[18:01:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:01:49] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.045, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=nan, n_estimators=581,
             n_jobs=-1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)